<a href="https://colab.research.google.com/github/nosark/ppo-lunar-lander-v2/blob/main/hugface_unit1_hands_on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies and create a virtual screen to view agent results
---

In [2]:
!apt install swig cmake


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.1).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 24 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (1,024 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubu

In [3]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 69.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of gymnasium[box2d] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 41.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 58.3 MB/s eta 0:00:00
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2349139 sha256=c356fc1135d08e1e9eef38b58f9a4a9c846a4e8f960a9f4909bb1f59a5af0f28
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf05

In [4]:
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [48.6 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,326 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/m



---



## Start Importing Packages
---

In [ ]:
import os

os.kill(os.getpid(), 9)

In [2]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [3]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import (
    notebook_login,
)

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor



### Test Environment to understand the problem
---

In [5]:
import gymnasium as gym

# Create enviroment for lunar lander
env = gym.make("LunarLander-v2")

# reset enviroment to start fresh
observation, info = env.reset()

for _ in range(20):
  # Take a random action
  action = env.action_space.sample()
  print("Action taken:", action)

  # Take action, and collect resulting: state, reward, terminated, truncated and info
  observation, reward, terminated, truncated, info = env.step(action)

  # If the game terminates (land and crash) or truncates(timeout)
if terminated or truncated:
  # Reset environment
  print("Environment is reset!")
  observation, info = env.reset()

env.close()

Action taken: 2
Action taken: 0
Action taken: 2
Action taken: 3
Action taken: 0
Action taken: 3
Action taken: 0
Action taken: 3
Action taken: 3
Action taken: 3
Action taken: 1
Action taken: 3
Action taken: 3
Action taken: 2
Action taken: 0
Action taken: 3
Action taken: 1
Action taken: 3
Action taken: 0
Action taken: 0


## Create Lunar Lander Enviroment
---

In [6]:
# Create the LunarLander enviroment
env = gym.make("LunarLander-v2")
env.reset()

print("---- OBSERVATION SPACE ---- \n")
print("Observation Space Shape", env.observation_space.shape)
# Get a random Observation
print("Sample Observation", env.observation_space.sample())

---- OBSERVATION SPACE ---- 

Observation Space Shape (8,)
Sample Observation [44.285652   29.004873    4.675746   -4.1811924  -2.6179893   0.30858412
  0.817402    0.93763036]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Observation Space Details
Observation Space Shape (8)

###### [HPC, VPC, HS, VS, Ang, AngSpeed, rightLegContacted, leftLegContacted]
---
- Horizontal Pad Coordinate(x)
- Vertical Pad Coordinate(y)
- Horizontal Speed(x)
- Vertical Speed(y)
- Angle
- Angular Speed
- If the left leg contact point has touched ground
- If the right leg contact point has touched ground

In [7]:
print("\n ---- ACTION SPACE ----\n")
print("Action Space Shape", env.action_space.n)
# Get random action space sample
print("Action Space Sample", env.action_space.sample())


 ---- ACTION SPACE ----

Action Space Shape 4
Action Space Sample 2


### Action Space Details
Action Space Shape (4)

[0, 1, 2, 3]
---
- Action 0 : Do nothing
- Action 1 : Fire left orientation engine
- Action 2 : Fire main engine
- Action 3 : Fire right orientation engine

#### Stacking 16 enviroments to improve diverity of events during training

In [8]:
env = make_vec_env("LunarLander-v2", n_envs=16)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
env = Monitor(gym.make("LunarLander-v2"))

# Create the agent
model = PPO(
    policy="MlpPolicy",
    env=env,
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=1,
)

model.learn(total_timesteps=1000000)

model_name="ppo-LunarHopefullyLander-v2"
model.save(model_name)


Streaming output truncated to the last 5000 lines.
|    value_loss           | 2.17e+03      |
-------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 339          |
|    ep_rew_mean          | 246          |
| time/                   |              |
|    fps                  | 546          |
|    iterations           | 740          |
|    time_elapsed         | 1385         |
|    total_timesteps      | 757760       |
| train/                  |              |
|    approx_kl            | 0.0047197025 |
|    clip_fraction        | 0.0215       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.783       |
|    explained_variance   | 0.979        |
|    learning_rate        | 0.0003       |
|    loss                 | 3.58         |
|    n_updates            | 2956         |
|    policy_gradient_loss | -0.00348     |
|    value_loss           | 17.3         |
-

### Evaluate the model

In [17]:
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean reward={mean_reward:.2f} +/- {std_reward}")


mean reward=259.72 +/- 38.89134589687146


#### Post to Hugging Face

In [18]:
notebook_login()
!git config --global credential.helper store

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub
env_id = "LunarLander-v2"
model_architecture = "PPO"
repo_id = "nosark/ppo-lunar-lander-v2"
commit_message= "Upload PPO LunarLander-v2 trained agent to HuggingFace"
# Create the evaluation env and set the render_mode="rgb_array"
eval_env = DummyVecEnv([lambda: Monitor(gym.make(env_id, render_mode="rgb_array"))])

# PLACE the package_to_hub function you've just filled here
package_to_hub(
    model=model,  # Our trained model
    model_name=model_name,  # The name of our trained model
    model_architecture=model_architecture,  # The model architecture we used: in our case PPO
    env_id=env_id,  # Name of the environment
    eval_env=eval_env,  # Evaluation Environment
    repo_id=repo_id,  # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
    commit_message=commit_message,
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmpm3purqci/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpm3purqci/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpm3purqci/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpm3purqci/-step-0-to-step-1000.mp4
ℹ Pushing repo nosark/ppo-lunar-lander-v2 to the Hugging Face Hub


policy.optimizer.pth:   0%|          | 0.00/88.4k [00:00<?, ?B/s]

policy.pth:   0%|          | 0.00/43.8k [00:00<?, ?B/s]

pytorch_variables.pth:   0%|          | 0.00/864 [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

ppo-LunarHopefullyLander-v2.zip:   0%|          | 0.00/147k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/nosark/ppo-lunar-lander-v2/tree/main/


'https://huggingface.co/nosark/ppo-lunar-lander-v2/tree/main/'